# 2025-11-23 ML 공부 정리 및 검증

오늘 공부한 내용:
1. Matrix Gradient (행렬 미분)
2. SGD와 Backpropagation
3. SVM (Hard/Soft Margin)

## 1. Matrix Gradient 기초

### 핵심 규칙
- 스칼라 함수를 벡터/행렬로 미분할 때, **미분 결과의 shape은 미분 변수의 shape과 같아야 함**
- `Ax` 형태에서:
  - `A`는 행렬 (m×n)
  - `x`는 벡터 (n×1)
  - 결과: (m×1)

### Case 1: Ax (A는 행, x는 열)
- `∂(Ax)/∂A = x^T` ✓ (맞음)
- `∂(Ax)/∂x = A^T` ✓ (맞음)

## 2. SGD와 Loss Function 미분

### MSE Loss
$$L = \sum (y - \hat{y})^2 = \sum (y - (wx + b))^2$$

### ❌ 틀린 부분 수정
**원래 작성:** `∂L/∂w = 2∑(ŷ-y)x^T`

**올바른 식:** `∂L/∂w = -2∑(y-ŷ)x^T = 2∑(ŷ-y)x^T` ✓

- w는 (1×n) 행벡터
- x는 (n×1) 열벡터
- ∂L/∂w의 shape은 (1×n)이어야 함
- 따라서 x를 전치하여 x^T (1×n) 형태로 만듦

### Logistic Regression (Cross Entropy)
$$L = -\sum [y\log(\sigma(wx+b)) + (1-y)\log(1-\sigma(wx+b))]$$

**미분 결과:**
$$\frac{\partial L}{\partial w} = \sum (\sigma(wx+b) - y)x^T$$ ✓ (맞음)

## 3. MLP Backpropagation

### 4층 신경망 (Input → Hidden1 → Hidden2 → Output)

**Forward:**
- z1 = w1·x
- a1 = τ(z1)
- z2 = w2·a1
- a2 = τ(z2)
- z3 = w3·a2
- o = σ(z3)

### ❌ 틀린 부분 수정

**SGD (single sample) - 원래 작성:**
- `∂L/∂w3 = (o-y)τ'(z3)z2^T`
- `∂L/∂w2 = σ3·τ'(z2)z1^T`
- `∂L/∂w1 = σ2·τ'(z1)x^T`

**올바른 식:**
- `∂L/∂w3 = (o-y)·σ'(z3)·a2^T` ✓ (output은 sigmoid)
- `∂L/∂w2 = δ3·w3^T·τ'(z2)·a1^T` where δ3=(o-y)·σ'(z3)
- `∂L/∂w1 = δ2·w2^T·τ'(z1)·x^T`

**Softmax + Cross Entropy:**
- `∂L/∂w3 = (o-y)·a2^T` ✓ (맞음, gradient는 단순화됨)

### Mini-batch/Full-batch Gradient

**❌ 틀린 부분:**
원래: `∂L/∂w3 = z2^T·(o-y)·τ'(z3)`

**✓ 올바른 식:**
- Batch size = N
- A2: (N × hidden_dim) - 배치의 activation
- O: (N × output_dim)
- Y: (N × output_dim)

$$\frac{\partial L}{\partial w_3} = A_2^T \cdot (O - Y) \cdot \sigma'(Z_3)$$

Shape: (hidden_dim × N) · (N × output_dim) = (hidden_dim × output_dim) ✓

## 4. SVM

### Hard Margin

**목적함수:**
$$\min_{w,b} \frac{1}{2}\|w\|^2$$
$$\text{s.t. } y_i(w^Tx_i + b) \geq 1$$

### ❌ 개념 오류 수정

**오해했던 부분:**
> "거리 d/||w||에서 d에 c를 곱해서 1로 만들면 w도 바뀌지만 1/||w||는 안바뀜"

**올바른 이해:**
- Margin = 2/||w||
- d = y(w^Tx + b) - 이것이 functional margin
- Functional margin을 1로 normalize하는 것은 **scaling 자유도를 제거**하기 위함
- w를 스케일링해도 결정 경계는 동일 (w, 2w, 3w 모두 같은 hyperplane)
- **제약식을 ≥1로 고정**함으로써 unique solution 가능

**Lagrangian:**
$$L = \frac{1}{2}\|w\|^2 - \sum_i \lambda_i[y_i(w^Tx_i+b) - 1]$$

### Soft Margin SVM

**Slack variable ξ:**

**목적함수:**
$$\min_{w,b,\xi} \frac{1}{2}\|w\|^2 + C\sum_i \xi_i$$

**제약:**
$$y_i(w^Tx_i + b) \geq 1 - \xi_i$$
$$\xi_i \geq 0$$

### Slack Variable 해석 ✓ (맞음)
- **ξ = 0:** 정확히 분류됨, margin 밖
- **0 < ξ < 1:** margin 안에 있지만 올바른 쪽
- **ξ > 1:** 잘못 분류됨 (hyperplane 넘어감)

### C 파라미터 ✓ (맞음)
- **C가 클수록:** 오분류 페널티 큼 → margin 좁음, overfitting 위험
- **C가 작을수록:** 오분류 허용 → margin 넓음, underfitting 가능

### ❌ 표현 수정
원래: "SVS가 좁게/넓게 잡힘"
올바름: "Support Vectors (SVs)가 적어짐/많아짐, Margin이 좁아짐/넓어짐"

## 연습 문제

### 문제 1: Matrix Gradient
다음을 증명하시오.

$$f(x) = x^TAx \text{ (A는 대칭 행렬)}$$
$$\frac{\partial f}{\partial x} = 2Ax$$

**Hint:** Chain rule 사용, x^TAx를 전개

In [ ]:
import numpy as np

# 수치적 검증
A = np.array([[2, 1], [1, 3]])
x = np.array([[1], [2]])

# f(x) = x^T A x
f = x.T @ A @ x
print(f"f(x) = {f[0,0]}")

# Gradient = 2Ax
grad = 2 * A @ x
print(f"∇f = \n{grad}")

# 수치 미분으로 검증
eps = 1e-5
numerical_grad = np.zeros_like(x, dtype=float)
for i in range(len(x)):
    x_plus = x.copy().astype(float)
    x_plus[i] += eps
    x_minus = x.copy().astype(float)
    x_minus[i] -= eps
    numerical_grad[i] = ((x_plus.T @ A @ x_plus) - (x_minus.T @ A @ x_minus)) / (2*eps)

print(f"\n수치 미분 결과:\n{numerical_grad}")
print(f"\n오차: {np.max(np.abs(grad - numerical_grad))}")

### 문제 2: Backpropagation

2층 신경망이 다음과 같이 주어졌을 때:
- Input: x = [1, 2]
- w1 = [[0.5, 0.3], [0.2, 0.4]] (2×2)
- w2 = [[0.6], [0.7]] (2×1)
- Activation: ReLU
- Loss: MSE, y = 1

∂L/∂w1과 ∂L/∂w2를 계산하시오.

In [ ]:
# Forward pass
x = np.array([[1], [2]])
w1 = np.array([[0.5, 0.3], [0.2, 0.4]])
w2 = np.array([[0.6], [0.7]])
y = 1

# Forward
z1 = w1 @ x
a1 = np.maximum(0, z1)  # ReLU
z2 = w2.T @ a1
output = z2[0, 0]

print(f"z1 = \n{z1}")
print(f"a1 = \n{a1}")
print(f"output = {output}")

# Loss
loss = 0.5 * (output - y)**2
print(f"Loss = {loss}")

# Backward
# ∂L/∂z2
dL_dz2 = output - y
print(f"\n∂L/∂z2 = {dL_dz2}")

# ∂L/∂w2 = a1 * ∂L/∂z2
dL_dw2 = a1 * dL_dz2
print(f"∂L/∂w2 = \n{dL_dw2}")

# ∂L/∂a1 = w2 * ∂L/∂z2
dL_da1 = w2 * dL_dz2

# ∂L/∂z1 = ∂L/∂a1 * ReLU'(z1)
dL_dz1 = dL_da1 * (z1 > 0)
print(f"∂L/∂z1 = \n{dL_dz1}")

# ∂L/∂w1 = ∂L/∂z1 @ x^T
dL_dw1 = dL_dz1 @ x.T
print(f"∂L/∂w1 = \n{dL_dw1}")

### 문제 3: SVM Margin 계산

다음 데이터가 주어졌을 때:
- Class +1: (2, 3), (3, 3)
- Class -1: (1, 1), (2, 1)

결정 경계가 w = [1, 1], b = -3일 때:
1. Margin 폭을 계산하시오
2. Support Vector를 찾으시오
3. Hard margin 조건을 만족하는지 확인하시오

In [ ]:
# 데이터
X_pos = np.array([[2, 3], [3, 3]])
X_neg = np.array([[1, 1], [2, 1]])
w = np.array([1, 1])
b = -3

# 1. Margin 계산
margin = 2 / np.linalg.norm(w)
print(f"Margin = 2/||w|| = {margin:.4f}")

# 2. 각 점의 functional margin 계산
print("\n각 점의 y(w^Tx + b):")
for i, x in enumerate(X_pos):
    val = w @ x + b
    print(f"  +1 class, point {i+1}: {val:.2f}")
    
for i, x in enumerate(X_neg):
    val = -(w @ x + b)  # y = -1
    print(f"  -1 class, point {i+1}: {val:.2f}")

# 3. Support Vector 찾기 (margin = 1인 점들)
print("\nSupport Vectors (y(w^Tx + b) = 1):")
for i, x in enumerate(X_pos):
    if np.abs(w @ x + b - 1) < 0.01:
        print(f"  {x}")
for i, x in enumerate(X_neg):
    if np.abs(-(w @ x + b) - 1) < 0.01:
        print(f"  {x}")

### 문제 4: Soft Margin과 C 파라미터

다음 상황을 분석하시오:
- 어떤 점의 slack variable ξ = 0.5
- C = 1일 때와 C = 10일 때 이 점이 목적함수에 미치는 영향 비교

In [ ]:
xi = 0.5

# C = 1
penalty_C1 = 1 * xi
print(f"C=1일 때 페널티: {penalty_C1}")

# C = 10  
penalty_C10 = 10 * xi
print(f"C=10일 때 페널티: {penalty_C10}")

print(f"\n페널티 비율: {penalty_C10/penalty_C1}배")
print("\n해석:")
print("- C가 클수록 오분류/margin 위반에 대한 페널티가 커짐")
print("- C=10: 더 엄격하게 분류, margin 좁음, overfitting 위험")
print("- C=1: 여유있게 분류, margin 넓음, generalization 좋음")

## 정리

### 오늘 수정한 주요 오류:
1. ✅ Backpropagation에서 활성화 함수 미분 누락 수정
2. ✅ Batch gradient의 행렬 곱셈 순서 수정
3. ✅ SVM에서 거리/margin 개념 명확화
4. ✅ C 파라미터의 역할 정확히 이해

### 핵심 원칙:
- **행렬 미분**: 결과의 shape = 미분 변수의 shape
- **Transpose**: 차원 맞추기 위해 사용
- **Backprop**: Chain rule 철저히 적용
- **SVM**: Margin 최대화 = ||w|| 최소화